In [1]:
import sys
sys.path.append("../..")

In [2]:
# Ensure the datasketch package is installed for running this notebook
!pip install datasketch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
import time
from tqdm import tqdm

Set paths to preprocessed train, test datasets \
**Prerequisite**: run preprocess_save_df.ipynb

In [9]:
PATH_TO_TRAIN_DATA = "../../bigdata2023classification/preprocessed_data/preprocessed_train_df.pkl"
PATH_TO_TEST_DATA = "../../bigdata2023classification/preprocessed_data/preprocessed_test_df.pkl"

In [10]:
train_df = pd.read_pickle(PATH_TO_TRAIN_DATA)
test_df = pd.read_pickle(PATH_TO_TEST_DATA)  

In [11]:
train_df.head(5)

,Id,Title,Content,Label
0,227464,netflix come cable box amazon grocery overlord,subscribe one three rinkydink comparatively sp...,Entertainment
1,244074,pharrell iranian president react tehran happy ...,pharrell iranian president react tehran happy ...,Entertainment
2,60707,wildlife service seek comment,fish wildlife service reopen comment period ad...,Technology
3,27883,facebook team storyful launch fb newswire,nature social medium mean often source real ti...,Technology
4,169596,caesar plan mln new york casino,caesar plan mln new york casino jul newsdesk l...,Business


In [12]:
test_df.head(5)

,Id,Title,Content
0,262120,tracy morgan upgrade fair condition crash,actor comedian tracy morgan upgrade fair condi...
1,175132,smartphones weigh samsung electronics guidance...,samsung electronics co ltd tuesday issue unexp...
2,218739,fbi denies fumble testimony xmen director brya...,michael egan iii say press conference thursday...
3,253483,bachelorette spoiler week recap eric hill drama,mixed emotion happen tonight onthe bachelorett...
4,224109,barack obama honour frankie knuckle letter lov...,president barack obama pay special tribute clu...


In [8]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Brute-Force Method

In [9]:
# Compute jaccard similarity for each pair of test and train documents
# return a dictionary maping test indexes to the k most similar train indexes regarding jaccard similarity of documents, 
# build time, query time 
def brute_force_search(test_docs, train_docs, k, threshold=0.8):
    start_time = time.time()
    results = {}
    for test_idx, test_doc in tqdm(test_docs.items(), desc="Brute Force Search - Processing Documents"):
        similarities = []
        for train_idx, train_doc in train_docs.items():
            sim = jaccard_similarity(test_doc, train_doc)
            if sim > threshold:
                similarities.append((train_idx, sim))

        # Sort by similarity and select top K
        similarities.sort(key=lambda x: x[1], reverse=True)
        results[test_idx] = similarities[:k]
    return results, time.time() - start_time


#  LSH with MinHash

In [10]:
def create_minhash(doc_shingles, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for shingle in doc_shingles:
        m.update(shingle.encode('utf8'))
    return m

In [1]:
# Compute jaccard similarity with LSH for each pair of test and train documents
# return a dictionary maping test indexes to the k most similar train indexes regarding jaccard similarity of documents, 
# build time, query time 
def lsh_process(train_docs, test_docs, num_perm, k=15, threshold=0.8):
    start_build_time = time.time()

    # Create and build LSH index
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    train_minhashes = {idx: create_minhash(shingles, num_perm) for idx, shingles in train_docs.items()}
    for idx, minhash in tqdm(train_minhashes.items(), desc="Building LSH Index"):
        lsh.insert(idx, minhash)

    end_build_time = time.time()
    build_time = end_build_time - start_build_time

    start_query_time = time.time()

    # Querying LSH index
    results = {}
    for test_idx, test_shingles in test_docs.items():
        test_minhash = create_minhash(test_shingles, num_perm)
        candidate_idxs = lsh.query(test_minhash)
        candidate_docs = {idx: train_docs[idx] for idx in candidate_idxs}
        similarities = [(idx, jaccard_similarity(test_shingles, shingles)) 
                        for idx, shingles in candidate_docs.items()]
        similarities.sort(key=lambda x: x[1], reverse=True)
        results[test_idx] = similarities[:k]

    end_query_time = time.time()
    query_time = end_query_time - start_query_time

    return results, build_time, query_time


# Apply and Evaluate each Method 

Create sets of shingles (unique words) from question

In [13]:
# Convert content into sets of words (Shingles)
train_df['Shingles'] = train_df['Content'].apply(lambda x: set(x.split()))
test_df['Shingles'] = test_df['Content'].apply(lambda x: set(x.split()))

train_doc_sets = train_df['Shingles'].to_dict()
test_doc_sets = test_df['Shingles'].to_dict()

# Number of permutations to test
permutations = [16, 32, 64]

# Number of nearest neighbors to find
k=15

In [14]:
# Function to calculate the fraction of correct similar documents
def calculate_fraction_correct(brute_force_results, lsh_results):
    correct_matches = 0
    total_matches = 0
    for test_idx, true_similar in brute_force_results.items():
        true_similar_ids = set(idx for idx, _ in true_similar)
        lsh_similar_ids = set(idx for idx, _ in lsh_results.get(test_idx, []))
        correct_matches += len(true_similar_ids.intersection(lsh_similar_ids))
        total_matches += len(true_similar_ids)
    return correct_matches / total_matches if total_matches > 0 else 0

In [15]:
print(f"train_df shape: {train_df.shape}")
print(f"test_df shape: {test_df.shape}")

train_df shape: (111795, 5)
test_df shape: (47912, 4)


In [16]:
evaluation_results = []

In [17]:
# Brute Force Method
print("Starting brute-force method...")
brute_force_results, brute_force_time = brute_force_search(test_doc_sets, train_doc_sets, k)
evaluation_results.append({
    "Type": "Brute-Force-Jaccard",
    "BuildTime": 0,
    "QueryTime": brute_force_time,
    "TotalTime": brute_force_time,
    "FractionCorrect": 1.0,  # Brute-force method is assumed to be 100% accurate
    "Parameters": "-"
})
results_df = pd.DataFrame(evaluation_results)
display(results_df)

Starting brute-force method...


Brute Force Search - Processing Documents:   0%|          | 0/47912 [00:00<?, ?it/s]

Brute Force Search - Processing Documents: 100%|██████████| 47912/47912 [25:27:02<00:00,  1.91s/it]    


,Type,BuildTime,QueryTime,TotalTime,FractionCorrect,Parameters
0,Brute-Force-Jaccard,0,91622.349267,91622.349267,1.0,-


In [18]:
# LSH Method for different permutations
for num_perm in permutations:
    print(f"Processing LSH with {num_perm} permutations...")
    lsh_results, build_time, query_time = lsh_process(train_doc_sets, test_doc_sets, num_perm, k)
    total_time = build_time + query_time
    fraction_correct = calculate_fraction_correct(brute_force_results, lsh_results, k)
    evaluation_results.append({
        "Type": "LSH-Jaccard",
        "BuildTime": build_time,
        "QueryTime": query_time,
        "TotalTime": total_time,
        "FractionCorrect": fraction_correct,
        "Parameters": f"Perm={num_perm}"
    })


Processing LSH with 16 permutations...


Building LSH Index: 100%|██████████| 111795/111795 [00:04<00:00, 23711.64it/s]


Processing LSH with 32 permutations...


Building LSH Index: 100%|██████████| 111795/111795 [00:05<00:00, 21610.72it/s]


Processing LSH with 64 permutations...


Building LSH Index: 100%|██████████| 111795/111795 [00:04<00:00, 23334.06it/s]


In [19]:
# Convert the results to a DataFrame
results_df = pd.DataFrame(evaluation_results)
display(results_df)

,Type,BuildTime,QueryTime,TotalTime,FractionCorrect,Parameters
0,Brute-Force-Jaccard,0.000000,91622.349267,91622.349267,1.000000,-
1,LSH-Jaccard,247.413489,90.840445,338.253934,0.703982,Perm=16
2,LSH-Jaccard,221.311057,84.337102,305.648159,0.708700,Perm=32
3,LSH-Jaccard,238.683804,108.599689,347.283494,0.773949,Perm=64


# Comments

**Efficiency vs. Accuracy Trade-off**: LSH significantly improves computational efficiency over brute-force methods for calculating Jaccard similarity, with a trade-off in accuracy. As the number of permutations increases, the accuracy of LSH approaches that of brute-force, but with diminishing returns in terms of time efficiency.